<a href="https://colab.research.google.com/github/annabatman2/Website/blob/main/Chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

from google.colab import files
uploaded = files.upload()




Saving intents.json to intents.json


In [ ]:
import random
import json
import pickle
import numpy as np
import nltk
from nltk.stem import WordNetLemmatizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout
from tensorflow.keras.optimizers.legacy import SGD
nltk.download('punkt')
nltk.download('wordnet')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [ ]:
lemmatizer = WordNetLemmatizer()
intents = json.loads(open('intents.json').read())

words = []
classes = []
documents = []
ignore = ['?','!',',','.']

for intent in intents['intents']:
    for pattern in intent['patterns']:
        word_list = nltk.word_tokenize(pattern)
        words.extend(word_list)
        documents.append((word_list, intent['tag']))
        if intent['tag'] not in classes:
          classes.append(intent['tag'])
print(documents)

words = [lemmatizer.lemmatize(word) for word in words if word not in ignore]
words = sorted(set(words))

classes = sorted(set(classes))
pickle.dump(words, open('words.pkl', 'wb'))
pickle.dump(classes, open('classes.pkl', 'wb'))

training = []
output_empty = [0]* len(classes)

# representing words as values by setting individual word values depending on if it occurs in a particular pattern
for document in documents:
    bag = []
    word_patterns = document[0]
    word_patterns = [lemmatizer.lemmatize(word.lower()) for word in word_patterns]
    for word in words:
        bag.append(1) if word in word_patterns else bag.append(0)

    output_row = list(output_empty)
    output_row[classes.index(document[1])] = 1
    training.append([bag, output_row])

random.shuffle(training)
training = np.array(training, dtype = object)

train_x = list(training[:, 0]) # This is the input (questions).
train_y = list(training[: ,1]) # This is the output (answers).

model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation = 'relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]),activation='softmax'))

sgd = SGD(learning_rate =0.01, decay = 1e-6, momentum = 0.9, nesterov = True)
model.compile(loss='categorical_crossentropy', optimizer = sgd, metrics=['accuracy'])

hist = model.fit(np.array(train_x), np.array(train_y), epochs = 200, batch_size = 5, verbose = 1)
model.save('ChatBotModel.h5', hist)








[(['hello'], 'greetings'), (['how', 'are', 'you'], 'greetings'), (['hi'], 'greetings'), (['whats', 'up'], 'greetings'), (['school', 'is', 'hard'], 'academic stress'), (['I', 'have', 'so', 'much', 'work'], 'academic stress'), (['I', 'have', 'an', 'exam', 'coming', 'up'], 'academic stress'), (['My', 'classes', 'are', 'hard'], 'academic stress'), (['I', 'am', 'tired'], 'tired'), (['I', 'just', 'want', 'to', 'sleep'], 'tired'), (['I', 'need', 'a', 'nap'], 'tired'), (['I', 'am', 'sad'], 'emotions'), (['I', 'feel', 'stressed'], 'emotions'), (['I', 'feel', 'upset'], 'emotions')]
Epoch 1/200
3/3 [==============================] - 1s 8ms/step - loss: 1.4720 - accuracy: 0.2143
Epoch 2/200
3/3 [==============================] - 0s 5ms/step - loss: 1.3464 - accuracy: 0.2857
Epoch 3/200
3/3 [==============================] - 0s 5ms/step - loss: 1.3763 - accuracy: 0.1429
Epoch 4/200
3/3 [==============================] - 0s 5ms/step - loss: 1.3314 - accuracy: 0.2857
Epoch 5/200
3/3 [================

In [ ]:
import random
import json
import pickle
import numpy as np
import nltk
from nltk.stem import WordNetLemmatizer
from tensorflow.keras.models import load_model
import tkinter as tk


In [ ]:
lemmatizer = WordNetLemmatizer()
#intents = json.loads(open('intents.json').read())

words = pickle.load(open('words.pkl', 'rb'))
classes = pickle.load(open('classes.pkl', 'rb'))
model = load_model('ChatBotModel.h5')

def clean_up(sentence):
  sentence_words = nltk.word_tokenize(sentence)
  sentence_words = [lemmatizer.lemmatize(word) for word in sentence_words]
  return sentence_words

def bag_of_words(sentence):
  sentence_words = clean_up(sentence)
  bag = [0]*len(words)
  for w in sentence_words:
    for i, word in enumerate(words):
      if word == w:
        bag[i] = 1
  return np.array(bag)

def predict_class(sentence):
  bow = bag_of_words(sentence)
  result = model.predict(np.array([bow]))[0]
  print(result)
  ERROR_THRESHOLD = 0.25
  result = [[i,r] for i, r in enumerate(result) if r > ERROR_THRESHOLD]

  result.sort(key = lambda x: x[1], reverse = True)
  return_list = []

  for r in result:
    return_list.append({'intent': classes[r[0]], 'probability': str(r[1])})
  return return_list

def get_response(intents_list, intents_json):
  tag = intents_list[0]['intent']
  list_of_intents = intents_json['intents']
  for i in list_of_intents:
    if i['tag'] == tag:
      result = random.choice(i['responses'])
      break
  return result

print("bot is running")
while True:
  message = input("")
  ints = predict_class(message)
  result = get_response(ints, intents)
  print(result)













hello
1/1 [==============================] - 0s 76ms/step
[1.5853919e-04 1.2604649e-04 9.9952626e-01 1.8917669e-04]
How can I help cheer you up?
i am stressed
1/1 [==============================] - 0s 28ms/step
[1.4984162e-03 9.9370265e-01 3.5920634e-04 4.4398811e-03]
Brighter days are ahead, keep moving forward and things will get better!
are you sure
1/1 [==============================] - 0s 34ms/step
[2.93646677e-04 1.11615045e-04 9.99494314e-01 1.00452249e-04]
Hello!


KeyboardInterrupt: ignored